By: Denver Chernin <br>

In [1]:
#Installing necessary package for data access
import sys
!conda install --yes --prefix {sys.prefix} pandasql

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pandasql


The following NEW packages will be INSTALLED:

  pandasql           conda-forge/noarch::pandasql-0.7.3-pyhd8ed1ab_0

The following packages will be UPDATED:

  ca-certificates                      2021.5.30-ha878542_0 --> 2021.10.8-ha878542_0
  certifi                          2021.5.30-py39hf3d152e_0 --> 2021.10.8-py39hf3d152e_0
  openssl                                 1.1.1k-h7f98852_1 --> 1.1.1l-h7f98852_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [2]:
#Importing necessary libraries
import pandas as p #dataframes to hold tables
from pandasql import sqldf #sql to dataframe conversion for data access
from sklearn.model_selection import train_test_split #method to split dataframe into train and test sets

In [3]:
#Lambda function to make sql queries of data frames cleaner
pysqldf = lambda q: sqldf(q, globals())

In [4]:
#Reading in the college data
df_college = p.read_csv('../data/CollegePlaying.csv')

#Reading in the batting data of all players
df_batting = p.read_csv('../data/Batting.csv')

#Reading in the pitching data for all pitchers
df_pitchers = p.read_csv('../data/Pitching.csv')

#Reading in the people data (allows me to translate playerIDs into names (first and last))
df_people = p.read_csv('../data/People.csv')

#Reading in the wOBA (weighted on base average) weights for every year to use in calculations 
#as the linear equations for wOBA stays the same every year, but the weights do not
df_weights = p.read_csv('../data/wOBA_Weights.csv')

In [55]:
#Grabbing the unique playerIDs that exist in the df_college table (AKA obtaining what players played in college)
df_college_playerids = pysqldf("SELECT DISTINCT playerID from df_college where yearID >= 1955 group by playerID")

In [56]:
df_college_playerids.head(5)

,playerID
0,aardsda01
1,abadan01
2,abbotje01
3,abbotji01
4,abbotky01


In [57]:
#Grabbing all of the batting statistics for each player that went to college
df_batters_college = pysqldf("SELECT * FROM df_batting WHERE df_batting.playerID IN (SELECT * FROM df_college_playerids) and G >= 50")

In [58]:
df_batters_college.head(5)

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,kindaje01,1957,1,CHN,NL,72,181,18,29,3,...,12.0,1.0,0.0,8,48.0,0.0,0.0,3.0,0.0,4.0
1,altmage01,1959,1,CHN,NL,135,420,54,103,14,...,47.0,1.0,0.0,34,80.0,4.0,7.0,6.0,1.0,8.0
2,cashno01,1959,1,CHA,AL,58,104,16,25,0,...,16.0,1.0,1.0,18,9.0,3.0,5.0,1.0,2.0,0.0
3,fairlro01,1959,1,LAN,NL,118,244,27,58,12,...,23.0,0.0,4.0,31,29.0,2.0,1.0,6.0,2.0,3.0
4,hadleke01,1959,1,KC1,AL,113,288,40,73,11,...,39.0,1.0,2.0,24,74.0,0.0,1.0,2.0,3.0,6.0


In [59]:
#Grabbing all of the batting statistics for each player that DIDNT go to college
df_batters_no_college = pysqldf("SELECT * FROM df_batting WHERE df_batting.playerID NOT IN (SELECT * FROM df_college_playerids) and G >= 50 and yearID > 1955")

In [60]:
df_batters_no_college.head(5)

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,aaronha01,1956,1,ML1,NL,153,609,106,200,34,...,92.0,2.0,4.0,37,54.0,6.0,2.0,5.0,7.0,21.0
1,adcocjo01,1956,1,ML1,NL,137,454,76,132,23,...,103.0,1.0,0.0,32,86.0,6.0,1.0,11.0,2.0,11.0
2,amorosa01,1956,1,BRO,NL,114,292,53,76,11,...,58.0,3.0,4.0,59,51.0,6.0,1.0,8.0,1.0,4.0
3,aparilu01,1956,1,CHA,AL,152,533,69,142,19,...,56.0,21.0,4.0,34,63.0,2.0,1.0,14.0,1.0,9.0
4,ashburi01,1956,1,PHI,NL,154,628,94,190,26,...,50.0,10.0,1.0,79,45.0,3.0,5.0,6.0,1.0,4.0


In [61]:
no_college_count = pysqldf("SELECT COUNT(*) as C from df_batters_no_college")
print("Number of games played >= 50, no college: " + str(no_college_count['C'].iloc[0]))

Number of games played >= 50, no college: 15769


In [62]:
college_count = pysqldf("SELECT COUNT(*) as C from df_batters_college where G >= 50")
print("Number of games played >= 50, college: " + str(college_count['C'].iloc[0]))

Number of games played >= 50, college: 11087


In [63]:
df_combined = p.concat([df_batters_college, df_batters_no_college])
count = pysqldf("SELECT COUNT(*) AS c FROM df_combined")
print("Number of total players: " + str(count['c'].iloc[0]))

Number of total players: 26856


$$OPS  = \frac{AB * (H + BB + HBP) + TB * (AB + BB + SF + HBP}{AB * (AB + BB + SF + HBP)}$$
<br>
$$wOBA = \frac{x*BB + x*HBP + x*1B + x*2B + x*3B + x*HR}{AB + BB - IBB + SF + HBP}$$

In [52]:
def season_averager_and_add_college_flag_and_tb(lst_rows):
    #Take each column that will be used in wOBA and OPS and average them over the rows
        #Columns needed for wOBA = uBB (labeled BB in dataset), HBP, 1B (labeled H in dataset), 2B, 3B, HR, AB, IBB, SF, HBP
        #Columns needed for OPS = H, BB, HBP, AB, SF, TB (total bases = 1*H + 2*2B + 3*3B + 4*HR)

    #Input: list of rows for one playerID
    #Output: a row with averaged statistics for that playerID along with a 2 new columns: one which is college flag indicating whether
    #or not that playerID has gone to college, the other is the total bases for that player
    ret = []
    ret.append(lst_rows[0][0]) # maintaing the playerID
    for r in lst_rows:
        print(r['H'])
        

In [69]:
for index, row in df_combined.iterrows():
    print(row)
    break

playerID    kindaje01
yearID           1957
stint               1
teamID            CHN
lgID               NL
G                  72
AB                181
R                  18
H                  29
2B                  3
3B                  0
HR                  6
RBI              12.0
SB                1.0
CS                0.0
BB                  8
SO               48.0
IBB               0.0
HBP               0.0
SH                3.0
SF                0.0
GIDP              4.0
Name: 0, dtype: object
